In [1]:
!nvidia-smi

Wed Jul 23 14:45:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 571.96                 Driver Version: 571.96         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              3W /   60W |      71MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
import ultralytics

In [4]:
from ultralytics import YOLO


In [5]:
#defining CBAM/SE module

import torch
import torch.nn as nn

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

class CBAM(nn.Module):
    def __init__(self, channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        # Channel attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False)
        )
        self.sigmoid_channel = nn.Sigmoid()

        # Spatial attention
        self.conv_spatial = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid_spatial = nn.Sigmoid()

    def forward(self, x):
        b, c, h, w = x.size()

        # Channel attention
        avg = self.avg_pool(x).view(b, c)
        max = self.max_pool(x).view(b, c)
        out = self.fc(avg) + self.fc(max)
        out = self.sigmoid_channel(out).view(b, c, 1, 1)
        x = x * out

        # Spatial attention
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        spatial = torch.cat([avg_out, max_out], dim=1)
        spatial = self.sigmoid_spatial(self.conv_spatial(spatial))
        x = x * spatial

        return x


In [6]:
#Loading and Patching SE Block and CBAM

from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
#print(model.model)

#for i, m in enumerate(model.model.model):
 #   print(f"[{i}] {m}")

#patching SE Vlock after first C2f 
model.model.model[2] = nn.Sequential(
    model.model.model[2],
    SEBlock(channels=32)
)

#CBAM after 2nd C2F
model.model.model[4] = nn.Sequential(
    model.model.model[4],
    CBAM(channels=64)
)

#for i, m in enumerate(model.model.model):
#    print(f"[{i}] {m}")

In [7]:
#Saving initial random patched model weights
torch.save(model.model.state_dict(), 'yolov8n_cbam_se_init.pt')


In [10]:
#Dataset
dataset_path = r'C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\dataset\Preprocessed\BDD100K Night time.yolov8'  

In [12]:
#Training
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # or yolov8s.pt, yolov8m.pt
model.train(data= r'C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\dataset\Preprocessed\BDD100K Night time.yolov8\data.yaml', epochs=100, imgsz=640)


New https://pypi.org/project/ultralytics/8.3.169 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.165  Python-3.9.13 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\dataset\Preprocessed\BDD100K Night time.yolov8\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.

train: Scanning C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\dataset\Preprocessed\BDD100K Night time.yolov8\train_augmented\labels.cache... 4452 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4452/4452 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access  (ping: 0.20.0 ms, read: 37.310.6 MB/s, size: 92.2 KB)


val: Scanning C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\dataset\Preprocessed\BDD100K Night time.yolov8\val\labels.cache... 385 images, 0 backgrounds, 0 corrupt: 100%|██████████| 385/385 [00:00<?, ?it/s]


...

                   all        385       4351      0.454      0.357      0.364      0.197



100 epochs completed in 2.012 hours.
Optimizer stripped from C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\runs\detect\train3\weights\best.pt, 6.2MB

Validating C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\runs\detect\train3\weights\best.pt...
Ultralytics 8.3.165  Python-3.9.13 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:08<00:00,  1.45it/s]


                   all        385       4351      0.503      0.385      0.381      0.215
                  bike         20         23      0.555      0.348      0.307      0.179
                   bus        385        449      0.712      0.728      0.742      0.514
                   car        385       3248      0.627      0.639      0.669      0.389
                 motor         13         14          0          0    0.00392    0.00167
                person        132        597       0.47      0.394      0.377      0.148
                 rider         19         20      0.656        0.2      0.185     0.0553
Speed: 0.4ms preprocess, 2.4ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\runs\detect\train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001FD4FEEC4F0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [ ]:
#Testing
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # or yolov8s.pt, yolov8m.pt
model.test(data= r'C:\Users\Sanya\OneDrive\Document\projects\YOLOv8BDD100k\dataset\Preprocessed\BDD100K Night time.yolov8\data.yaml', epochs=100, imgsz=640)
